In [14]:
import os
import pandas as pd

# Define the directory where the Excel files are stored
folder_path = r'Dataset\train'

In [15]:
dataframes = []


In [33]:

# Folder where your CSV files are stored
folder_path = 'Dataset/train'

# List of CSV files (only filenames, not full paths)
files = ['ex_1.csv', 'ex_20.csv', 'ex_21.csv', 'ex_24.csv', 'ex_9.csv', 'ex_23.csv']
dataframes = {}

# Loop through each file and read it into a dataframe
for file in files:
    file_path = os.path.join(folder_path, file)  # Create the full path to the file
    df = pd.read_csv(file_path, encoding='ascii')  # Use the full file path
    dataframes[file] = df  # Add dataframe to the dictionary
    
    # Convert 'time' column to numeric (if it's not already)
    df['time'] = pd.to_numeric(df['time'], errors='coerce')
    
    # Print information about each dataframe
    print(f"{file}:")
    print(f"Time range: {df['time'].min()} to {df['time'].max()}")
    print(f"Number of rows: {len(df)}")
    
    # Ensure there's no missing data in 'time' before calculating step
    if df['time'].isna().sum() == 0:
        print(f"Time step: {df['time'].diff().mode().values[0]:.2f}")
    else:
        print("Warning: There are missing values in the 'time' column. Time step calculation skipped.")
    
    print()

# Calculate total number of rows
total_rows = sum(len(df) for df in dataframes.values())
print(f"Total number of rows across all datasets: {total_rows}")
print(f"Target number of rows: 71225")
print(f"Missing rows: {71225 - total_rows}")


ex_1.csv:
Time range: 810.07028 to 10720.65858
Number of rows: 9920
Time step: 1.00

ex_20.csv:
Time range: 849.22 to 7824.0
Number of rows: 6495
Time step: 1.00

ex_21.csv:
Time range: 847.72 to 7776.6
Number of rows: 6495
Time step: 1.00

ex_24.csv:
Time range: 811.04 to 10579.6
Number of rows: 9023
Time step: 1.00

ex_9.csv:
Time range: 818.84 to 12636.84
Number of rows: 11819
Time step: 1.00

ex_23.csv:
Time range: 810.0 to 10954.52
Number of rows: 9188
Time step: 1.00

Total number of rows across all datasets: 52940
Target number of rows: 71225
Missing rows: 18285


In [56]:
import os
import pandas as pd

# Folder where your CSV files are stored
folder_path = 'Dataset/train'

# List of CSV files (only filenames, not full paths)
files = ['ex_1.csv', 'ex_20.csv', 'ex_21.csv', 'ex_24.csv', 'ex_9.csv', 'ex_23.csv']
dataframes = []

# Step 1: Load datasets into DataFrames
for file in files:
    file_path = os.path.join(folder_path, file)  # Create the full path to the file
    df = pd.read_csv(file_path, encoding='ascii')  # Read the CSV file
    df['time'] = pd.to_numeric(df['time'], errors='coerce')  # Ensure 'time' is numeric
    
    # Add the DataFrame to the list
    dataframes.append(df)

# Step 2: Create a complete time series covering the expected range
full_time_series = pd.concat([df['time'] for df in dataframes]).dropna().unique()

# Step 3: Initialize a DataFrame for merged results
merged_df = pd.DataFrame({'time': full_time_series})

# Step 4: Merge all DataFrames into the merged_df based on 'time'
for df in dataframes:
    # Ensure that only the columns we want to merge are considered
    relevant_columns = df.columns[df.columns != 'time']  
    # Set the 'time' column as the index for the current DataFrame
    df.set_index('time', inplace=True)

    # For each relevant column, initialize it in merged_df if not already present
    for column in relevant_columns:
        # Check if the column already exists in merged_df; if not, initialize it
        if column not in merged_df.columns:
            merged_df[column] = None  # Create the column if it doesn't exist
        merged_df[column] = merged_df[column].combine_first(df[column])

# Step 5: Check the shape of the merged DataFrame
print("Shape of merged DataFrame:", merged_df.shape)

# Step 6: Display the first few rows of the merged DataFrame
print(merged_df.head())

# Optional: Save the merged DataFrame to a CSV file
merged_df.to_csv('merged_dataset.csv', index=False)


Shape of merged DataFrame: (51706, 3)
        time input_voltage el_power
0  810.07028          None     None
1  811.06938          None     None
2  812.06848          None     None
3  813.06758          None     None
4  814.06668          None     None


In [57]:
print(df.dtypes)

input_voltage    float64
el_power         float64
dtype: object
